In [ ]:
import math

teaspoons = {'tablespoons':1/3,'cups':1/48,'pint':1/96,'quart':1/192,'gallon':1/768}
tablespoon = {'teaspoons':3,'cups':1/16,'pint':1/32,'quart':1/64,'gallon':1/256}
cup = {'teaspoons':48,'tablespoons':16,'pint':1/2,'quart':1/4,'gallon':1/16}
pint = {'teaspoons':96,'tablespoons':32,'cup':2,'quart':1/2,'gallon':1/8}
quart = {'teaspoons':192,'tablespoons':64,'cup':4,'pint':2,'gallon':1/4}
gallon = {'teaspoons':768,'tablespoons':256,'cup':16,'pint':8,'quart':4}




In [1]:
import pandas as pd

x = pd.DataFrame()

print(type(x)==str)

False


In [16]:
import pathlib
x = './erer/wewwe/qwqwqw/wowcsv'
print(pathlib.Path(x).resolve().stem)
print(pathlib.Path(x).absolute()+'sss')


wowcsv


TypeError: unsupported operand type(s) for +: 'WindowsPath' and 'str'

accordion__arrow-icon tmg-svg-icon
accordion__trigger site-header__accordion-trigger
advert js-advert advert--mpu
advert js-advert js-advert-banner is-sticky advert--banner advert--header
advert-label advert-label--sidebar
all-section__selector grid-col grid-col-12 grid-col-md-5 grid-col-lg-4
all-sections grid
all-sections-country-selector__icon-flag
all-sections-country-selector__icon-flag-link
all-sections-country-selector__wrapper
all-sections-modal all-sections-modal--hidden
all-sections__arrow-icon accordion__arrow-icon tmg-svg-icon
all-sections__heading all-sections__main-heading u-heading-2
all-sections__heading all-sections__more-heading u-heading-3
all-sections__item
all-sections__item all-sections__item--has-children accordion accordion--sm accordion--is-collapsed
all-sections__link
all-sections__link all-sections__link--is-primary
all-sections__link-heading
all-sections__list
all-sections__more-container grid-col grid-col-12 grid-col-md-5 grid-col-lg-4
all-sections__more-ite